# Day 9: BERT 微调与模型部署

本讲义演示如何在 **GLUE MRPC** 任务上微调 BERT 模型，以及如何将训练好的模型上传到 **Hugging Face Hub** 与他人分享和使用。

**学习目标：**
- 实践完整的 BERT 微调工作流
- 理解 Hugging Face Hub 的模型管理与分享机制
- 掌握 `huggingface_hub` 库的使用（`upload_file`, `Repository`, `push_to_hub`）
- 学习传统 git 工作流与自动化部署的对比

**关键概念：**
- **微调（Fine-tuning）**：在预训练模型基础上，用特定任务数据进行训练
- **MRPC 任务**：判断两个句子是否为同义句（句子对分类）
- **Hub 部署**：将模型版本控制、分享、协作的中心化平台

**NOTE**：BERT 微调通常需要 GPU。如果无 GPU，建议在 Google Colab 或 HuggingFace Spaces 上运行。

## 使用 scripts 中的完整实现

**完整的 BERT 微调脚本已在 `scripts/day17.py` 中实现**

该脚本包含：
- 数据加载与预处理
- 模型初始化（带随机输出层）
- 使用 Trainer 进行微调
- 在测试集上评估
- 参数详解

**为什么在 CPU 上无法运行？**
- BERT-base 有 110M 参数，微调时需要存储梯度、优化器状态等
- 最小显存需求通常在 6-8 GB
- CPU 微调可能性极小，性能无法接受

**推荐环境：**
- 本地 GPU（NVIDIA RTX 2080 或更高）
- 云服务（Google Colab Free Tier、AWS SageMaker）
- Hugging Face Spaces（免费 CPU 环境有限制）

## 模型部署到 Hugging Face Hub

完成训练后，通常希望将模型分享给团队或社区。Hugging Face Hub 提供三种部署方式：

**方案 1：自动推送（集成在 Trainer 中）**
- 优点：最简便，一行配置
- 缺点：功能相对固定

**方案 2：使用 `huggingface_hub` 库**
- `upload_file()`：适合小文件（< 5 GB）
- `Repository` 类：适合大文件与复杂协作

**方案 3：传统 git 工作流**
- 克隆 hub 上的仓库到本地
- 进行标准 git 操作（add、commit、push）
- 完全控制版本历史

**建议：**
- 快速实验：方案 1
- 生产部署：方案 3（完整版本控制）

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## 方案 1：Trainer 自动推送

在 `TrainingArguments` 中添加 `push_to_hub=True`，Trainer 会在训练结束后自动上传模型。

```python
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc",           # 输出目录（也是 Hub 上的仓库名）
    push_to_hub=True,                # 启用自动推送
    save_strategy="epoch",           # 每个 epoch 保存一次
    hub_model_id="dove667/my-model"  # Hub 上的模型 ID（可选）
)

trainer = Trainer(...)
trainer.train()

# 训练完成后自动上传到 Hub
```

**前置条件**：需要通过 `huggingface-cli login` 进行身份验证

## 使用huggingface_hub


## 使用 huggingface-cli 登录

在终端执行：

```bash
huggingface-cli login
```

然后输入 Hugging Face 账户的 **Access Token**（可从 https://huggingface.co/settings/tokens 获取）

验证成功后，本机的 Hugging Face 工具就可以访问你的账户。

## 方案 2a：使用 `upload_file` 上传小文件

**适用场景**：模型大小 < 5 GB

```python
from huggingface_hub import create_repo, upload_file

# 1. 在 Hub 上创建仓库
create_repo("my-model", exist_ok=True)

# 2. 上传文件
upload_file(
    path_or_fileobj="path/to/local/model.bin",    # 本地文件路径
    path_in_repo="pytorch_model.bin",              # Hub 上的文件路径
    repo_id="dove667/my-model"                     # 仓库 ID（用户名/仓库名）
)
```

**优点**：简单直接
**缺点**：不支持大文件和版本控制

In [ ]:
from huggingface_hub import create_repo,upload_file

In [8]:
create_repo("lstm-emotion-detection", exist_ok=True)

RepoUrl('https://huggingface.co/Dove667/lstm-emotion-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='Dove667/lstm-emotion-detection')

In [12]:
upload_file(
    path_or_fileobj="/home/dove/DLModel/SimpleLSTM/checkpoint_9.pth",
    path_in_repo="model.pth",
    repo_id="dove667/lstm-emotion-detection",
)

checkpoint_9.pth:   0%|          | 0.00/48.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Dove667/lstm-emotion-detection/commit/cd2ff86aac8b8b58a67252c04075f298b5c0392e', commit_message='Upload model.pth with huggingface_hub', commit_description='', oid='cd2ff86aac8b8b58a67252c04075f298b5c0392e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dove667/lstm-emotion-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='Dove667/lstm-emotion-detection'), pr_revision=None, pr_num=None)

## 方案 2b：使用 `Repository` 类处理大文件

**适用场景**：模型大小 > 5 GB，需要版本控制

**前置条件**：
```bash
# 安装 git-lfs（Large File Storage）用于处理大文件
sudo apt-get install git-lfs
git lfs install
```

**工作流程**：
```python
from huggingface_hub import Repository

# 1. 克隆 Hub 仓库到本地
repo = Repository(
    local_dir="./my-model",
    clone_from="dove667/my-model"
)

# 2. 本地操作
repo.git_pull()
# ... 保存模型、编辑 README 等 ...

# 3. 提交并推送到 Hub
repo.git_add()
repo.git_commit("Add trained model checkpoint")
repo.git_push()
```

**优点**：
- 支持大文件
- 完整版本控制
- 适合团队协作

**缺点**：
- 相对复杂
- 需要 git-lfs

In [ ]:
from huggingface_hub import Repository

repo = Repository("<path_to_dummy_folder>", clone_from="<namespace>/dummy-model")

然后可以调用一系列git方法：
```python
repo.git_pull()

model.save_pretrained("<path_to_dummy_folder>")
tokenizer.save_pretrained("<path_to_dummy_folder>")

repo.git_add()
repo.git_commit("Add model and tokenizer files")
repo.git_push()

## 方案 3：传统 git 工作流

**最灵活，推荐用于生产部署**

**第一次设置**：
```bash
# 克隆仓库
git clone https://huggingface.co/dove667/my-model
cd my-model
```

**日常开发**：
```bash
# 开发、训练...

# 保存模型
model.save_pretrained(".")
tokenizer.save_pretrained(".")

# 更新 README 等文档
# ...

# 标准 git 操作
git add .
git commit -m "Update model with new training run"
git push
```

**优点**：
- 完全标准化的 git 工作流
- 支持所有 git 特性（分支、标签、历史）
- 适合团队协作
- 与 CI/CD 集成方便

**缺点**：
- 相对复杂
- 需要 git-lfs 处理大文件

对模型任意操作，比如微调：
```python
from transformers import AutoModelForMaskedLM, AutoTokenizer

checkpoint = "camembert-base"

model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Do whatever with the model, train it, fine-tune it...

model.save_pretrained("<path_to_dummy_folder>")
tokenizer.save_pretrained("<path_to_dummy_folder>")
```

## 总结与建议

| 方案 | 文件大小 | 复杂度 | 版本控制 | 推荐场景 |
|------|--------|--------|---------|---------|
| Trainer 自动推送 | 任意 | ⭐ | ✅ | 快速实验、研究 |
| upload_file | < 5 GB | ⭐⭐ | ❌ | 一次性上传小模型 |
| Repository | > 5 GB | ⭐⭐⭐ | ✅ | 大模型、高级用户 |
| 传统 git | 任意 | ⭐⭐⭐ | ✅✅ | 生产部署、团队协作 |

**快速选择指南：**
1. **第一次用**？ → 使用 Trainer 自动推送
2. **小模型**？ → 使用 `upload_file`
3. **大模型或需要版本控制**？ → 使用 Repository 或 git
4. **生产环境或团队合作**？ → 使用传统 git

下一步：学习如何从 Hub 加载模型进行推理、如何创建 Model Card 描述、如何设置 Hub 的自动测试等。